In [1]:
import os
import sys
# #file_dir = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
# print(file_dir)
# sys.path.append(file_dir)

import torch
import numpy as np
import pandas as pd  
import torch.nn as nn
import argparse
import configparser
from datetime import datetime
from model.AGCRN import AGCRN as Network
from model.BasicTrainer import Trainer
from lib.TrainInits import init_seed
from lib.dataloader import get_dataloader
from lib.TrainInits import print_model_parameters


In [2]:
import sys
sys.argv=['']
del sys

In [4]:
data = np.load('data/PEMS04/pems04.npz')['data'][:, :, 0]  

In [4]:
import torch
# Define the path to the adjacency matrix
adj_file_name = "est_adj_agcrn/E_init.npy"

# Check if the file exists and load it
if os.path.exists(adj_file_name):
    # Load the matrix from the file
    precomputed_embeddings = torch.tensor(np.load(adj_file_name), dtype=torch.float32)
    print(f"Loaded precomputed embeddings from {adj_file_name} with shape {precomputed_embeddings.shape}")
else:
    precomputed_embeddings = None
    print(f"Embedding file not found at {adj_file_name}; using random initialization.")


Loaded precomputed embeddings from est_adj_agcrn/E_init.npy with shape torch.Size([307, 10])


In [6]:
Mode = 'Train'
DEBUG = 'True'
DATASET = 'PEMSD4'      #PEMSD4 or PEMSD8
DEVICE = 'cuda:0'
MODEL = 'AGCRN'

#get configuration
config_file = 'model/PEMSD4_AGCRN.conf'  # Assuming the file is in the same directory as your script
config = configparser.ConfigParser()
config.read(config_file)

from lib.metrics import MAE_torch
def masked_mae_loss(scaler, mask_value):
    def loss(preds, labels):
        if scaler:
            preds = scaler.inverse_transform(preds)
            labels = scaler.inverse_transform(labels)
        mae = MAE_torch(pred=preds, true=labels, mask_value=mask_value)
        return mae
    return loss

In [7]:
#parser
args = argparse.ArgumentParser(description='arguments')
args.add_argument('--dataset', default=DATASET, type=str)
args.add_argument('--mode', default=Mode, type=str)
args.add_argument('--device', default=DEVICE, type=str, help='indices of GPUs')
args.add_argument('--debug', default=DEBUG, type=eval)
args.add_argument('--model', default=MODEL, type=str)
args.add_argument('--cuda', default=True, type=bool)
#data
args.add_argument('--val_ratio', default=config['data']['val_ratio'], type=float)
args.add_argument('--test_ratio', default=config['data']['test_ratio'], type=float)
args.add_argument('--lag', default=config['data']['lag'], type=int)
args.add_argument('--horizon', default=config['data']['horizon'], type=int)
args.add_argument('--num_nodes', default=config['data']['num_nodes'], type=int)
args.add_argument('--tod', default=config['data']['tod'], type=eval)
args.add_argument('--normalizer', default=config['data']['normalizer'], type=str)
args.add_argument('--column_wise', default=config['data']['column_wise'], type=eval)
args.add_argument('--default_graph', default=config['data']['default_graph'], type=eval)
#model
args.add_argument('--input_dim', default=config['model']['input_dim'], type=int)
args.add_argument('--output_dim', default=config['model']['output_dim'], type=int)
args.add_argument('--embed_dim', default=config['model']['embed_dim'], type=int)
args.add_argument('--rnn_units', default=config['model']['rnn_units'], type=int)
args.add_argument('--num_layers', default=config['model']['num_layers'], type=int)
args.add_argument('--cheb_k', default=config['model']['cheb_order'], type=int)
#train
args.add_argument('--loss_func', default=config['train']['loss_func'], type=str)
args.add_argument('--seed', default=config['train']['seed'], type=int)
args.add_argument('--batch_size', default=config['train']['batch_size'], type=int)
args.add_argument('--epochs', default=config['train']['epochs'], type=int)
args.add_argument('--lr_init', default=config['train']['lr_init'], type=float)
args.add_argument('--lr_decay', default=config['train']['lr_decay'], type=eval)
args.add_argument('--lr_decay_rate', default=config['train']['lr_decay_rate'], type=float)
args.add_argument('--lr_decay_step', default=config['train']['lr_decay_step'], type=str)
args.add_argument('--early_stop', default=config['train']['early_stop'], type=eval)
args.add_argument('--early_stop_patience', default=config['train']['early_stop_patience'], type=int)
args.add_argument('--grad_norm', default=config['train']['grad_norm'], type=eval)
args.add_argument('--max_grad_norm', default=config['train']['max_grad_norm'], type=int)
args.add_argument('--teacher_forcing', default=False, type=bool)
#args.add_argument('--tf_decay_steps', default=2000, type=int, help='teacher forcing decay steps')
args.add_argument('--real_value', default=config['train']['real_value'], type=eval, help = 'use real value for loss calculation')
#test
args.add_argument('--mae_thresh', default=config['test']['mae_thresh'], type=eval)
args.add_argument('--mape_thresh', default=config['test']['mape_thresh'], type=float)
#log
args.add_argument('--log_dir', default='./', type=str)
args.add_argument('--log_step', default=config['log']['log_step'], type=int)
args.add_argument('--plot', default=config['log']['plot'], type=eval)
args = args.parse_args()
init_seed(args.seed)
if torch.cuda.is_available():
    torch.cuda.set_device(int(args.device[5]))
else:
    args.device = 'cpu'



In [8]:
#init model
# Initialize the model with precomputed embeddings
model = Network(args, precomputed_embeddings=precomputed_embeddings)
model = model.to(args.device)
for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
    else:
        nn.init.uniform_(p)
print_model_parameters(model, only_num=False)

#load dataset
train_loader, val_loader, test_loader, scaler = get_dataloader(args,
                                                               normalizer=args.normalizer,
                                                               tod=args.tod, dow=False,
                                                               weather=False, single=False)



*****************Model Parameter*****************
node_embeddings torch.Size([307, 10]) True
encoder.dcrnn_cells.0.gate.weights_pool torch.Size([10, 2, 65, 128]) True
encoder.dcrnn_cells.0.gate.bias_pool torch.Size([10, 128]) True
encoder.dcrnn_cells.0.update.weights_pool torch.Size([10, 2, 65, 64]) True
encoder.dcrnn_cells.0.update.bias_pool torch.Size([10, 64]) True
encoder.dcrnn_cells.1.gate.weights_pool torch.Size([10, 2, 128, 128]) True
encoder.dcrnn_cells.1.gate.bias_pool torch.Size([10, 128]) True
encoder.dcrnn_cells.1.update.weights_pool torch.Size([10, 2, 128, 64]) True
encoder.dcrnn_cells.1.update.bias_pool torch.Size([10, 64]) True
end_conv.weight torch.Size([12, 1, 1, 64]) True
end_conv.bias torch.Size([12]) True
Total params num: 748810
*****************Finish Parameter****************
Load PEMSD4 Dataset shaped:  (16992, 307, 1) 919.0 0.0 211.7007794815878 180.0
Normalize the dataset by Standard Normalization
Train:  (10173, 12, 307, 1) (10173, 12, 307, 1)
Val:  (3375, 12

/home/jovyan/work/lib/dataloader.py:77: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:83.)
  X, Y = TensorFloat(X), TensorFloat(Y)


In [9]:
# Adjust embeddings to match expected node count
expected_nodes = args.num_nodes  # 307
current_nodes, embed_dim = precomputed_embeddings.shape  # 307, embed_dim
if current_nodes < expected_nodes:
    padding = torch.zeros(expected_nodes - current_nodes, embed_dim)  # Zero embeddings for missing nodes
    precomputed_embeddings = torch.cat((precomputed_embeddings, padding), dim=0)


In [10]:
import sys
__file__ = 'data/PEMS04/pems04.npz'
file_dir = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
print(file_dir)
sys.path.append(file_dir)

/home/jovyan/work/data


In [11]:
#DEVICE = 'cpu'  # Update the device to CPU


In [13]:
# if torch.cuda.is_available():
#     args.device = 'cuda:0'
# else:
#     print("CUDA is not available. Switching to CPU.")
#     args.device = 'cpu'

# model = model.to(args.device)

# # Update the trainer call to ensure data and model are on the correct device
# trainer = Trainer(model, loss, optimizer, train_loader, val_loader, test_loader, scaler, args, lr_scheduler=lr_scheduler)
# if args.mode.lower() == 'train':
#     trainer.train()
# elif args.mode.lower() == 'test':
#     model.load_state_dict(torch.load('../pre-trained/{}.pth'.format(args.dataset), map_location=args.device))
#     print("Load saved model")
#     trainer.test(model, trainer.args, test_loader, scaler, trainer.logger)
# else:
#     raise ValueError(f"Invalid mode: {args.mode}. Expected 'train' or 'test'.")


In [14]:
if torch.cuda.is_available():
    args.device = 'cuda:0'
else:
    print("CUDA is not available. Using CPU.")
    args.device = 'cpu'

# Ensure the model and data are moved to the correct device
model = model.to(args.device)


In [25]:
! nvidia-smi

Failed to initialize NVML: Unknown Error


In [17]:
Mode = 'train'  # Use lowercase 'train' instead of 'Train'

if args.mode.lower() == 'train':  # Normalize to lowercase for comparison
    trainer.train()
elif args.mode.lower() == 'test':
    model.load_state_dict(torch.load('../pre-trained/{}.pth'.format(args.dataset)))
    print("Load saved model")
    trainer.test(model, trainer.args, test_loader, scaler, trainer.logger)
else:
    raise ValueError(f"Invalid mode: {args.mode}. Expected 'train' or 'test'.")


2024-12-06 17:59: Train Epoch 1: 0/158 Loss: 207.048584
2024-12-06 17:59: Train Epoch 1: 20/158 Loss: 180.675720
2024-12-06 18:00: Train Epoch 1: 40/158 Loss: 195.901291
2024-12-06 18:00: Train Epoch 1: 60/158 Loss: 185.352875
2024-12-06 18:00: Train Epoch 1: 80/158 Loss: 183.701355
2024-12-06 18:00: Train Epoch 1: 100/158 Loss: 205.981171
2024-12-06 18:00: Train Epoch 1: 120/158 Loss: 194.591919
2024-12-06 18:00: Train Epoch 1: 140/158 Loss: 175.404388
2024-12-06 18:00: **********Train Epoch 1: averaged Loss: 191.398754, tf_ratio: 1.000000
2024-12-06 18:00: **********Val Epoch 1: average Loss: 192.045513
2024-12-06 18:00: *********************************Current best model saved!
2024-12-06 18:00: Train Epoch 2: 0/158 Loss: 183.595688
2024-12-06 18:00: Train Epoch 2: 20/158 Loss: 187.893127
2024-12-06 18:00: Train Epoch 2: 40/158 Loss: 165.947006
2024-12-06 18:00: Train Epoch 2: 60/158 Loss: 171.949448
2024-12-06 18:00: Train Epoch 2: 80/158 Loss: 151.457260
2024-12-06 18:00: Train Ep

In [ ]:
print(f"Using loss function: {args.loss_func}")


In [16]:
# #init loss function, optimizer
# if args.loss_func == 'mask_mae':
#     loss = masked_mae_loss(scaler, mask_value=0.0)
# elif args.loss_func == 'mae':
#     loss = torch.nn.L1Loss().to(args.device)
# elif args.loss_func == 'mse':
#     loss = torch.nn.MSELoss().to(args.device)
# else:
#     raise ValueError

# optimizer = torch.optim.Adam(params=model.parameters(), lr=args.lr_init, eps=1.0e-8,
#                              weight_decay=0, amsgrad=False)
# #learning rate decay
# lr_scheduler = None
# if args.lr_decay:
#     print('Applying learning rate decay.')
#     lr_decay_steps = [int(i) for i in list(args.lr_decay_step.split(','))]
#     lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer=optimizer,
#                                                         milestones=lr_decay_steps,
#                                                         gamma=args.lr_decay_rate)
#     #lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=64)

# #config log path
# current_time = datetime.now().strftime('%Y%m%d%H%M%S')
# current_dir = os.path.dirname(os.path.realpath(__file__))
# log_dir = os.path.join(current_dir,'experiments', args.dataset, current_time)
# args.log_dir = log_dir

# #start training
# trainer = Trainer(model, loss, optimizer, train_loader, val_loader, test_loader, scaler,
#                   args, lr_scheduler=lr_scheduler)
# if args.mode == 'train':
#     trainer.train()
# elif args.mode == 'test':
#     model.load_state_dict(torch.load('../pre-trained/{}.pth'.format(args.dataset)))
#     print("Load saved model")
#     trainer.test(model, trainer.args, test_loader, scaler, trainer.logger)
# else:
#     raise ValueError


2024-12-06 17:59: Experiment log path in: /home/jovyan/work/data/PEMS04/experiments/PEMSD4/20241206175947


ValueError: 

In [18]:
# import torch
# import numpy as np
# import os
# from datetime import datetime
# from model.AGCRN import AGCRN as Network
# from model.BasicTrainer import Trainer
# from lib.dataloader import get_dataloader
# from lib.TrainInits import print_model_parameters, init_seed
# from lib.metrics import MAE_torch

# # Setup configuration and argument parser
# args = argparse.Namespace()

# # Configuration Parameters
# args.dataset = 'PEMSD4'
# args.mode = 'train'  # Use 'train' or 'test'
# args.device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
# args.loss_func = 'mae'
# args.lr_init = 0.003
# args.lr_decay = True
# args.lr_decay_rate = 0.3
# args.lr_decay_step = '5,20,40,70'
# args.epochs = 100
# args.batch_size = 64
# args.weight_decay = 0.0001
# args.log_step = 20
# args.embed_dim = 10  # Example embedding dimension
# args.num_nodes = 307  # Example node count

# # Initialize seed
# init_seed(42)

# # Initialize the model
# precomputed_embeddings = np.random.rand(args.num_nodes, args.embed_dim).astype(np.float32)
# model = Network(args, precomputed_embeddings=torch.tensor(precomputed_embeddings))
# model = model.to(args.device)

# for p in model.parameters():
#     if p.dim() > 1:
#         torch.nn.init.xavier_uniform_(p)
#     else:
#         torch.nn.init.uniform_(p)
# print_model_parameters(model, only_num=False)

# # Load dataset
# train_loader, val_loader, test_loader, scaler = get_dataloader(
#     args, normalizer='std', tod=False, dow=False, weather=False, single=False
# )

# # Initialize loss function
# if args.loss_func == 'mask_mae':
#     def masked_mae_loss(scaler, mask_value):
#         def loss(preds, labels):
#             preds = scaler.inverse_transform(preds)
#             labels = scaler.inverse_transform(labels)
#             return MAE_torch(preds, labels, mask_value)
#         return loss

#     loss = masked_mae_loss(scaler, mask_value=0.0)
# elif args.loss_func == 'mae':
#     loss = torch.nn.L1Loss().to(args.device)
# elif args.loss_func == 'mse':
#     loss = torch.nn.MSELoss().to(args.device)
# else:
#     raise ValueError(f"Unknown loss function: {args.loss_func}")

# # Initialize optimizer
# optimizer = torch.optim.Adam(
#     params=model.parameters(), lr=args.lr_init, eps=1.0e-8, weight_decay=args.weight_decay, amsgrad=False
# )

# # Learning rate scheduler
# lr_scheduler = None
# if args.lr_decay:
#     print("Applying learning rate decay.")
#     lr_decay_steps = [int(i) for i in args.lr_decay_step.split(',')]
#     lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(
#         optimizer=optimizer, milestones=lr_decay_steps, gamma=args.lr_decay_rate
#     )

# # Configure log path
# current_time = datetime.now().strftime('%Y%m%d%H%M%S')
# current_dir = os.path.dirname(os.path.realpath(__file__))
# log_dir = os.path.join(current_dir, 'experiments', args.dataset, current_time)
# args.log_dir = log_dir
# os.makedirs(log_dir, exist_ok=True)
# print(f"Experiment log path in: {args.log_dir}")

# # Initialize trainer
# trainer = Trainer(model, loss, optimizer, train_loader, val_loader, test_loader, scaler, args, lr_scheduler)

# # Train or Test
# if args.mode.lower() == 'train':
#     trainer.train()
# elif args.mode.lower() == 'test':
#     model.load_state_dict(torch.load(f'../pre-trained/{args.dataset}.pth'))
#     print("Loaded saved model.")
#     trainer.test(model, args, test_loader, scaler, trainer.logger)
# else:
#     raise ValueError(f"Invalid mode: {args.mode}. Expected 'train' or 'test'.")


AttributeError: 'Namespace' object has no attribute 'input_dim'